In [ ]:
%run ./taskbacklog/analysis.ipynb

In [ ]:
from datetime import datetime, timedelta, timezone
from threading import Timer
from IPython.display import display, Math, Latex, Markdown
import pandas as pd
import os, time

class System():
    def __init__(self, name, tips):
        self.name = name
        self.tips = tips

systems = {'c': System(name="Calendar", tips="""
Can you shrink the “filled” blocks in your schedule? For example, can you:
* Shorten meetings? Achieve the meeting goal so it can be cancelled.
* Get exercise at the same time?
* "Work" less than 40 hours?
* Ignore a deadline to do weightier work instead? See [Decision: Should you respect a particular deadline?](https://docs.google.com/document/d/1JKY2lkoiNFfMR-UEcAL3EEOeIVhftIgdke-60MdYk64/edit).

Review:
* [Google Calendar - today](https://calendar.google.com/calendar/r/day)
* [Calendar - Outlook - today](https://outlook.office.com/calendar/view/day)
"""),
           'g': System(name="Gmail", tips="""
- [Gmail](https://mail.google.com/mail/u/0/#inbox) tips:
  - Use j/k
  - Find email count in the top right (to the left of the gear).
"""),
           'o': System(name="Outlook", tips="""
- [Outlook](https://outlook.office.com/mail/inbox) tips:
  - Use j/k!
  - Snooze on your phone
  - Find email count by selecting all conversations then deselecting one.
"""),
           'k': System(name="Keep", tips="""
- [Keep](https://keep.google.com/#home) tips:
  - Use j/k
"""),
           'u': System(name="Unfinished", tips="""
If you already started some task, have a bias towards finishing it before you move on (lower E[**E**] for tasks in working memory).

If you must move on and you also believe the task will come up again, move unorganized working memory into a git commit message (long-term shared storage). For example:
* Chrome tabs
* tmux tabs
* Other applications

Consider restarting your machine after a cleanup.
"""),
           's': System(name="Schedule", tips="""
Schedule the item to your day. See:
  * [Decision: Should you schedule your day down to the hour?](https://docs.google.com/document/d/1QuEe13p89Ffcq7Ao_zVcYyrHaq9rMvv8oOCKzmN9BnU/edit)
  * See [Process: Schedule day (home)](https://docs.google.com/document/d/1iPrL-jZtRQC2e3C70vFTNahcJa_H9IBMWOByTciPkHk/edit) (i.e. use a timer).
""")}

option_table = pd.DataFrame(
    {systems[system].name: pd.Series(dict({'Key': system})) for system in systems})

# It's probably best to estimate the "questionable" weight from the tasks
# on your backlog (opportunity costs). For now, hard code to an average.
questionable_weight = 5

def prompt_for_integer(prompt_string):
    expect_integer = None
    while(expect_integer is None):
        try:
            expect_integer = int(input(prompt_string))
        except ValueError:
            print("Please enter an integer.")
    return expect_integer

# https://stackoverflow.com/a/1301528/622049
#
# TODO: Set timezone in Docker so you can use python3 conversions:
# https://stackoverflow.com/a/13287083/622049
os.environ['TZ'] = 'America/Chicago'
time.tzset()

def start_timer(timebox, timeout_string):
    cutoff = (datetime.now() + timebox).strftime('%Y-%m-%d %H:%M')
    display(Markdown(f"""
- Timebox: {timebox}
- Cutoff: {cutoff}"""
))
    t = Timer(timebox.seconds, lambda: display(Markdown("""
```diff
- {}
```
""".format(timeout_string))))
    t.start()
    return t

# Skim online task systems (starting with the most interruptive)
while True:
    display(option_table)
    sk = input("Enter system selection (empty to stop): ")
    if sk not in systems:
        break

    system = systems[sk]
    display(Markdown(system.tips))
    
    if sk == 'c':
        input("Hit enter to continue.")
        continue

    if sk == 's':
        task = prompt_for_integer("Enter task index: ")
        if task < 0 or task >= full.size:
            break
        t = start_timer(timedelta(hours=full.at[task, "Timebox"]),
                        "Ask someone for help.")
        row = full.loc[task]
        
        if row.E > 1 and not row.url:
            display(Markdown(
"""
Before starting work on a task larger than an hour, report your plan:
- On a shared backlog
- At a daily stand-up
"""
        ))

        if row.E > 4:
            display(Markdown(
"""
Break down tasks larger than 4 hours. See:
- [Process: Backlog grooming](https://docs.google.com/document/d/1bmRN4n0kbMN2EOhMKk62VMjpkAYcolWbKj0vPLyJKa8/edit#)
- [Decision: Should you split story subtasks further?][1]
- [Process: PBI subtask planning](https://docs.google.com/document/d/1g39MM493y1KSkZLYHWtxmnf-68CZjfw_rPADFaErvgk/edit).

[1]: https://docs.google.com/document/d/1hQ99w3ZnrLpwygfZJHFTKfbrYqwo-cXB5tZIBmKtw4o/edit#
"""
        ))

        from datetime import timedelta
        if row.age < 3:
            display(Markdown(
"""
Avoid recently created stories. See:
- [Process: Handling interruptions](https://docs.google.com/document/d/1Y0LbIWeP4wnwm09FsC2YejIMFvrG3wfNtiVQVQaG4ew/edit)
- [What if the build breaks?](https://docs.google.com/document/d/1wp7nLk6tkN8FlLELK7-IcbWgNlpmw51MC1DWSW9_yLc/edit)
"""
        ))
        if row.age < timedelta(weeks=2).days:
            display(Markdown(
"""
This story was created in the last two weeks. Is it valuable because it is focal?
"""
        ))
        input("Hit enter to complete.")
        t.cancel()
        continue
        
    tasks = prompt_for_integer("Enter unsorted tasks in {}: ".format(system.name))
    if tasks == 0:
        continue

    t = start_timer(timedelta(minutes=2*tasks), "Timeout! Routine: Come back to this system.")
    print("")

    display(Markdown(
"""
Finish small tasks:
$$E[\\textbf{E}] < 1m$$
Move large tasks to your backlog:
$$E[\\textbf{E}] > 4m$$
Complete small tasks before starting medium tasks.
"""
))

    while True:
        medium = input("Does a medium-sized task remain (empty to stop)?")
        t.cancel()
        if not medium:
            break

        display(Markdown(
"""
### Limit TVE analysis timebox to 10% of E[**E**]:
1. [Testable][1]
  * When are you done?
  * Is this actionable yet?
  * How can you help?
    * Predict the content of an upcoming merge request?
    * Pair program or ask a question?
1. [Value][2] (r.v. **V**)?
  * It’s presumptuous to expect someone to sell value on every task or story. Ask them for their estimate of E[V].
  * Common value decisions:
    * [Decision: Should your team do independent code reviews?][22]
    * [Decision: Should you automate a manual task?][23]
    * [Decision: Are automatic tests a good investment?][24]
    * [Decision: Should you share library binaries?][25]
1. [Estimate][3] (r.v. **E**)?
  * Can you get E[**E**] smaller by bundling with other tasks? Examples:
    * Group items you do off your computer.
    * Combine notes. Copy the “super note” to an existing document.
    * Group everything you need to talk about with one person.
  * Common processes:
    * [Process: Code review checklist][31]
    * [Process: Investigate root cause][32]

[1]: https://docs.google.com/document/d/1FNKdVqkBM11VZrNuX00_IT_u5CNU02J1Ys1RQADTLEM/edit
[2]: https://docs.google.com/document/d/1UZhNMbYM4Pq46Hken1MwCbAU8rWYZT_T02eUiUoyqDg/edit
[3]: https://docs.google.com/document/d/1FMH80GUHQ_pwtuTLG0pmGsPZyXZh2CV1nSzRNBU42W0/edit#heading=h.718s8y7tnrt

[22]: https://docs.google.com/document/d/15tY5wjoEl2q5eW1lGn-RZyDSxGViGz_r8cN-owGU3BI/edit#
[23]: https://docs.google.com/document/d/1v-4nxZ_dNjkfdeR1Zy5LQehmpTqgBmHVC-G6fhLd4AA/edit
[24]: https://docs.google.com/document/d/1LJH2SERIA6SJj27oQk2_TW49T1XxPJ8QLmg0z0tpd2g/edit
[25]: https://docs.google.com/document/d/1Ed3T8hlYYZurQLvzBtVYHBs1OGGfiwy9XjZob8JHHUs/edit

[31]: https://docs.google.com/document/d/1vEsv-v6Br6Gg7fV0ES5QltMOWK5jadaaVYSJwHxh1IM/edit
[32]: https://davidvandebunte.gitlab.io/shared-notes/post/process-investigate-root-cause/
"""
))

        display(Markdown(
"""
Estimate the weight of the medium task: 
$$W = E[\\textbf{V}/\\textbf{E}]$$
"""
))
        weight = prompt_for_integer("Estimated weight: ")
        if weight < questionable_weight:
            display(Markdown(
"""
Be aware:
$$E[\\textbf{V}/\\textbf{E}] < W_{typical}$$
If this task appeared recently, snooze it to remove focal value.
"""
))

        t = start_timer(timedelta(minutes=8), "Timeout! Routine: Rotate back to this medium-sized task.")
        input("Hit enter to complete.")
        print("")